In [1]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

#Import our helper functions
import helper_functions

# Step 1: View the list of reports available

In [2]:
#View the list of precheck reports
report_lst = helper_functions.get_available_precheck_reports()

report_lst

,ReportName,ReportPath
64,Pre-Check Deduction,/content/folder[@name='zzzCompany Folders']/fo...
65,Pre-Check Earnings,/content/folder[@name='zzzCompany Folders']/fo...
66,Pre-Check Master,/content/folder[@name='zzzCompany Folders']/fo...
67,Pre-Check Payroll Register,/content/folder[@name='zzzCompany Folders']/fo...
68,Pre-Check Tax,/content/folder[@name='zzzCompany Folders']/fo...
69,Prenote,/content/folder[@name='zzzCompany Folders']/fo...


In [3]:
#report_path = report_lst[report_lst['ReportName'] == 'Pre-Check Deduction']['ReportPath'].loc[64]
report_path = helper_functions.get_report_file_path('Pre-Check Deduction')
report_path

"/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Pre-Check Deduction']"

## Step 2: Retrieve the report and export as a CSV

In [4]:
df = helper_functions.retrieve_report(report_path) 
df 

,period_control_date,employee_number,deduction/benefit_code,deduction/benefit_(stub),current_amount_(employee),current_amount_(employer)
0,2023-02-17T00:00:00,001002805,401K,401k,66.41,66.41
1,2023-02-17T00:00:00,001005225,401K,401k,29.32,73.3
2,2023-02-17T00:00:00,001008575,401KR,401k Roth,168.06,168.06
3,2023-02-17T00:00:00,001008575,DENTL,Dental Coverage,2.99,26.84
4,2023-02-17T00:00:00,001008575,MEDRX,Medical-PPO,64,293.5
5,2023-02-17T00:00:00,001008575,OFFC,Off-Cycle Check,1326.04,0
6,2023-02-17T00:00:00,001008663,OFFC,Off-Cycle Check,0,0
7,2023-02-17T00:00:00,001008693,OFFC,Off-Cycle Check,0,0
8,2023-02-17T00:00:00,001008575,VIS,Vision,0.52,4.65


In [5]:
#Remove the T's from the column names
df['period_control_date'] = df['period_control_date'].str.replace('T', ' ')

df

,period_control_date,employee_number,deduction/benefit_code,deduction/benefit_(stub),current_amount_(employee),current_amount_(employer)
0,2023-02-17 00:00:00,001002805,401K,401k,66.41,66.41
1,2023-02-17 00:00:00,001005225,401K,401k,29.32,73.3
2,2023-02-17 00:00:00,001008575,401KR,401k Roth,168.06,168.06
3,2023-02-17 00:00:00,001008575,DENTL,Dental Coverage,2.99,26.84
4,2023-02-17 00:00:00,001008575,MEDRX,Medical-PPO,64,293.5
5,2023-02-17 00:00:00,001008575,OFFC,Off-Cycle Check,1326.04,0
6,2023-02-17 00:00:00,001008663,OFFC,Off-Cycle Check,0,0
7,2023-02-17 00:00:00,001008693,OFFC,Off-Cycle Check,0,0
8,2023-02-17 00:00:00,001008575,VIS,Vision,0.52,4.65


In [6]:
df['current_amount_(employer)'] = df['current_amount_(employer)'].astype(str).replace('@xs:nil', '0')
df['current_amount_(employee)'] = df['current_amount_(employee)'].astype(str).replace('@xs:nil', '0')

df

,period_control_date,employee_number,deduction/benefit_code,deduction/benefit_(stub),current_amount_(employee),current_amount_(employer)
0,2023-02-17 00:00:00,001002805,401K,401k,66.41,66.41
1,2023-02-17 00:00:00,001005225,401K,401k,29.32,73.3
2,2023-02-17 00:00:00,001008575,401KR,401k Roth,168.06,168.06
3,2023-02-17 00:00:00,001008575,DENTL,Dental Coverage,2.99,26.84
4,2023-02-17 00:00:00,001008575,MEDRX,Medical-PPO,64,293.5
5,2023-02-17 00:00:00,001008575,OFFC,Off-Cycle Check,1326.04,0
6,2023-02-17 00:00:00,001008663,OFFC,Off-Cycle Check,0,0
7,2023-02-17 00:00:00,001008693,OFFC,Off-Cycle Check,0,0
8,2023-02-17 00:00:00,001008575,VIS,Vision,0.52,4.65


# Step 3: Export the report as a CSV

In [7]:
df.to_csv('ukg_bulk_insert.csv', index=False)

# Step 4: Insert the report into BiData2 as a table

In [8]:
helper_functions.create_sql_table_v2('STG_PreCheck_Deduction')

# Step 5: Close the API connection

In [9]:
helper_functions.close_api_connection()

'<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOffResponse</a:Action></s:Header><s:Body><LogOffResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOffResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId i:nil="true"/><ClientAccessKey>PLAB4</ClientAccessKey><Token i:nil="true"/><Status>LoggedOff</Status><StatusMessage i:nil="true"/><InstanceKey i:nil="true"/></LogOffResult></LogOffResponse></s:Body></s:Envelope>'